In [2]:
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load

In [3]:
time_precision = 4   # jagame päeva xh suurusteks plokkideks


X_train = pd.read_csv("../../data/X_train.csv")
X_train = X_train.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_train.ToimAeg = X_train.ToimAeg.values.astype('datetime64[D]')
X_train.ToimAeg = X_train['ToimAeg'].apply(lambda x: x.replace(hour=x.hour-x.hour%time_precision, minute=0, second=0))
X_train["Aasta"] = X_train['ToimAeg'].dt.year
X_train["Kuu"] = X_train['ToimAeg'].dt.month
X_train["Päev"] = X_train['ToimAeg'].dt.day
X_train["Tund"] = X_train['ToimAeg'].dt.hour
X_train.drop(columns="ToimAeg", inplace=True)
X_train = X_train.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_train.head()

y_train = pd.read_csv("../../data/y_train.csv")
y_train = y_train.apply(pd.to_numeric, errors='ignore', downcast='integer')
y_train.head()

X_valid = pd.read_csv("../../data/X_valid.csv")
X_valid = X_valid.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_valid.ToimAeg = X_valid.ToimAeg.values.astype('datetime64[D]')
X_valid.ToimAeg = X_valid['ToimAeg'].apply(lambda x: x.replace(hour=x.hour-x.hour%time_precision, minute=0, second=0))
X_valid["Aasta"] = X_valid['ToimAeg'].dt.year
X_valid["Kuu"] = X_valid['ToimAeg'].dt.month
X_valid["Päev"] = X_valid['ToimAeg'].dt.day
X_valid["Tund"] = X_valid['ToimAeg'].dt.hour
X_valid.drop(columns="ToimAeg", inplace=True)
X_valid = X_valid.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_valid.head()

y_valid = pd.read_csv("../../data/y_valid.csv")
y_valid = y_valid.apply(pd.to_numeric, errors='ignore', downcast='integer')
y_valid.head()

X_train.head()

,Lest_X,Lest_Y,KOHT_APTEEK,KOHT_AVALIK_KOHT,KOHT_BUROO_KONTOR,KOHT_BUSSIJAAM_BUSSITERMINAAL,KOHT_BUSSIPEATUS,KOHT_EHITUS,KOHT_HOTELL_MOTELL,KOHT_KASIINO,...,ToimNadalapaev_Kolmapäev,ToimNadalapaev_Laupäev,ToimNadalapaev_Neljapäev,ToimNadalapaev_Pühapäev,ToimNadalapaev_Reede,ToimNadalapaev_Teisipäev,Aasta,Kuu,Päev,Tund
0,6584000,540500,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,2017,2,5,0
1,6593000,557000,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,2019,6,6,0
2,6461000,663000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2012,11,19,0
3,6590000,539000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,2019,11,11,0
4,6590500,549500,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,2015,3,24,0


In [4]:
y_train.info()
y_train.drop(columns=["Kahjusumma","Koht"], inplace=True)
y_valid.drop(columns=["Kahjusumma", "Koht"], inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62476 entries, 0 to 62475
Data columns (total 16 columns):
LIIK_AVALIKU_KORRA_RIKKUMINE     62476 non-null int8
LIIK_JALGRATTA_MOPEEDI_VARGUS    62476 non-null int8
LIIK_KEHALINE_VAARKOHTLEMINE     62476 non-null int8
LIIK_KELMUS                      62476 non-null int8
LIIK_KONFLIKT/OMAVOLI            62476 non-null int8
LIIK_KOOLIVAGIVALD               62476 non-null int8
LIIK_LIIKLUSRIKKUMINE            62476 non-null int8
LIIK_MOOTORSOIDUKI_VARGUS        62476 non-null int8
LIIK_MUU                         62476 non-null int8
LIIK_PISIVARGUS                  62476 non-null int8
LIIK_ROOVIMINE                   62476 non-null int8
LIIK_TULEKAHJU                   62476 non-null int8
LIIK_VANDALISM                   62476 non-null int8
LIIK_VARGUS                      62476 non-null int8
Kahjusumma                       62476 non-null object
Koht                             62476 non-null object
dtypes: int8(14), object(2)
memory usag

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error as MSE

rfc = RandomForestClassifier(n_estimators=100, random_state=1).fit(X_train, y_train)
probs_rfc = rfc.predict_proba(X_valid)

In [8]:
df_proba = pd.DataFrame(np.array([x.transpose()[1] for x in probs_rfc]).transpose(), columns=y_valid.columns) # @rasmus ma ei oska kirjelda miks see rida vajalik aga igastahes see tuleb sellest probs_rfc kujust mis ei sobinud


#lihtsalt predict jaoks df_proba = pd.DataFrame(probs_rfc, columns=y_valid.columns) # @rasmus ma ei oska kirjelda miks see rida vajalik aga igastahes see tuleb sellest probs_rfc kujust mis ei sobinud
print(y_valid)
#.shape
df_proba
print(MSE(y_valid, df_proba))

#probs_rfc.transpose().shape

      LIIK_AVALIKU_KORRA_RIKKUMINE  LIIK_JALGRATTA_MOPEEDI_VARGUS  \
0                                1                              0   
1                                0                              0   
2                                0                              0   
3                                0                              0   
4                                0                              0   
...                            ...                            ...   
7804                             0                              0   
7805                             0                              0   
7806                             1                              0   
7807                             0                              0   
7808                             0                              0   

      LIIK_KEHALINE_VAARKOHTLEMINE  LIIK_KELMUS  LIIK_KONFLIKT/OMAVOLI  \
0                                0            0                      0   
1                      

In [23]:
def applyCutoff(df, cutoff):
    a = df.copy()

    for col in a.columns:
        a[col] = a[col].apply(lambda x: 1 if x > cutoff else 0)
    
    return a


In [37]:
def chooseBiggestProb(df):
    a = df.copy()
    for index, row in a.iterrows():
        maxindeks = row.idxmax()
        
        for i, item in row.iteritems():
            if i == maxindeks:
                row[i] = 1
            else:
                row[i] = 0
    print(a)

chooseBiggestProb(df_proba)

      LIIK_AVALIKU_KORRA_RIKKUMINE  LIIK_JALGRATTA_MOPEEDI_VARGUS  \
0                              1.0                            0.0   
1                              0.0                            0.0   
2                              0.0                            0.0   
3                              0.0                            0.0   
4                              1.0                            0.0   
...                            ...                            ...   
7804                           0.0                            0.0   
7805                           0.0                            0.0   
7806                           0.0                            0.0   
7807                           0.0                            0.0   
7808                           0.0                            0.0   

      LIIK_KEHALINE_VAARKOHTLEMINE  LIIK_KELMUS  LIIK_KONFLIKT/OMAVOLI  \
0                              0.0          0.0                    0.0   
1                      

In [12]:
print(df_proba)
from sklearn import metrics
print(metrics.roc_auc_score(y_valid, df_proba))
for i in df_proba.columns:
    print(y_valid[i])
    print(i)
    print(metrics.accuracy_score(y_valid[i], df_proba[i]), i, sep="\t")

      LIIK_AVALIKU_KORRA_RIKKUMINE  LIIK_JALGRATTA_MOPEEDI_VARGUS  \
0                            0.885                           0.00   
1                            0.000                           0.00   
2                            0.000                           0.01   
3                            0.000                           0.00   
4                            0.390                           0.01   
...                            ...                            ...   
7804                         0.000                           0.00   
7805                         0.010                           0.00   
7806                         0.020                           0.01   
7807                         0.000                           0.00   
7808                         0.000                           0.00   

      LIIK_KEHALINE_VAARKOHTLEMINE  LIIK_KELMUS  LIIK_KONFLIKT/OMAVOLI  \
0                            0.115        0.000                   0.00   
1                      

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [8]:
for i in df_proba.columns:
    print(sum(df_proba[i]), sum(y_valid[i]), i, sep="\t")

847.6463690475982	866	LIIK_AVALIKU_KORRA_RIKKUMINE
201.95896825396886	199	LIIK_JALGRATTA_MOPEEDI_VARGUS
258.2935357142852	259	LIIK_KEHALINE_VAARKOHTLEMINE
39.467726190476334	43	LIIK_KELMUS
24.413166666666825	33	LIIK_KONFLIKT/OMAVOLI
15.359999999999959	17	LIIK_KOOLIVAGIVALD
10.183833333333281	6	LIIK_LIIKLUSRIKKUMINE
43.653000000000134	39	LIIK_MOOTORSOIDUKI_VARGUS
305.09083730158346	291	LIIK_MUU
2367.4627362914616	2328	LIIK_PISIVARGUS
85.4265357142864	91	LIIK_ROOVIMINE
11.803166666666579	10	LIIK_TULEKAHJU
565.6696071428456	555	LIIK_VANDALISM
3727.725720057732	3787	LIIK_VARGUS


In [16]:
metrics.accuracy_score(y_valid, df_proba)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets

In [1]:
metrics.confusion_matrix(y_valid, df_proba)

NameError: name 'metrics' is not defined

In [19]:
dump(rfc, "rfc_minic.joblib", compress=0, protocol=5)
print("done dumping")

done dumping
